Import Libraries

In [1]:
import os 
import keras
import tensorflow as tf

# Data 
from tensorflow.keras import layers

# Model 
from keras.models import Sequential
from keras.layers import GlobalAvgPool2D as GAP, Dense, Dropout

# Callbacks 
from keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.preprocessing.image import ImageDataGenerator

Prepare Training Data Set

In [2]:
# Define data path
data_path = "D:\\AI\\data\\"

class_names = sorted(os.listdir(data_path))
n_classes = len(class_names)
class_dis = [len(os.listdir(data_path + name)) for name in class_names]

print(f"Total Number of Classes : {n_classes} \nClass Names : {class_names}")

# Create training and validation dataset from directory
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    data_path,
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=(224, 224),
    batch_size=128,
)

# Rescale dataset from 0-255 to 0-1
preprocessing = keras.Sequential([
    layers.Rescaling(1./255),
])

train_ds = train_ds.map(lambda x, y: (preprocessing(x), y))
val_ds = val_ds.map(lambda x, y: (preprocessing(x), y))

Total Number of Classes : 10 
Class Names : ['bedroom', 'bridge', 'church_outdoor', 'classroom', 'conference_room', 'dining_room', 'kitchen', 'living_room', 'restaurant', 'tower']
Found 3000 files belonging to 10 classes.
Using 2400 files for training.
Using 600 files for validation.


Load a pretrain model

In [2]:
base_model = keras.applications.MobileNetV2(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

base_model.trainable = False
base_model.summary()

9406464/9406464 [==============================] - 16s 2us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
               

Create new model on top

In [5]:
# New model architecture
name = "MobileNetV2"
model = Sequential([
    base_model,
    GAP(),
    Dense(256, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.2),
    Dense(n_classes, activation='softmax')
], name=name)

# Callbacks
cbs = [EarlyStopping(patience=3, restore_best_weights=True), ModelCheckpoint(name + ".h5", save_best_only=True)]

model.summary()

Model: "MobileNetV2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 2,588,490
Trainable params: 330,506
Non-t

Transfer Learning the Model

In [6]:
# Model 
opt = tf.keras.optimizers.Adam(learning_rate=2e-3)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Model Training
model.fit(train_ds, validation_data=val_ds, callbacks=cbs, epochs=8)

Epoch 1/8
19/19 [==============================] - 41s 2s/step - loss: 1.2443 - accuracy: 0.5612 - val_loss: 0.8420 - val_accuracy: 0.6967
Epoch 2/8
19/19 [==============================] - 38s 2s/step - loss: 0.6463 - accuracy: 0.7717 - val_loss: 0.7783 - val_accuracy: 0.7150
Epoch 3/8
19/19 [==============================] - 38s 2s/step - loss: 0.4884 - accuracy: 0.8200 - val_loss: 0.7449 - val_accuracy: 0.7283
Epoch 4/8
19/19 [==============================] - 38s 2s/step - loss: 0.3891 - accuracy: 0.8587 - val_loss: 0.7121 - val_accuracy: 0.7433
Epoch 5/8
19/19 [==============================] - 37s 2s/step - loss: 0.2890 - accuracy: 0.8971 - val_loss: 0.7714 - val_accuracy: 0.7333
Epoch 6/8
19/19 [==============================] - 37s 2s/step - loss: 0.2349 - accuracy: 0.9279 - val_loss: 0.8226 - val_accuracy: 0.7450
Epoch 7/8
19/19 [==============================] - 37s 2s/step - loss: 0.1814 - accuracy: 0.9458 - val_loss: 0.7799 - val_accuracy: 0.7450


Fine Tuned the Model

In [7]:
# Pre-Trained Model 
base_model.trainable = True

# Model 
opt = tf.keras.optimizers.Adam(learning_rate=1e-6)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Model Training
model.fit(train_ds, validation_data=val_ds, callbacks=cbs, epochs=4)


Epoch 1/6
19/19 [==============================] - 195s 9s/step - loss: 1.0858 - accuracy: 0.6313 - val_loss: 0.7143 - val_accuracy: 0.7433
Epoch 2/6
19/19 [==============================] - 171s 9s/step - loss: 1.0780 - accuracy: 0.6367 - val_loss: 0.7167 - val_accuracy: 0.7450
Epoch 3/6
19/19 [==============================] - 169s 9s/step - loss: 1.0405 - accuracy: 0.6529 - val_loss: 0.7192 - val_accuracy: 0.7417
Epoch 4/6
19/19 [==============================] - 168s 9s/step - loss: 1.0308 - accuracy: 0.6458 - val_loss: 0.7218 - val_accuracy: 0.7433


Save The Model 

In [8]:
model.save(name + ".h5", save_format='h5')